# Part 9: Random Forest regression model

The potency of the novel generated compounds for the BACE1 receptor will be determined with machine learning models. In this project will two different models be compared; a random forest regression model and a neuron network model. These models will be trained and validated with the filtered ChEMBL bioactivity data from part 3. In this part will the random forest regression model be trained and validated, followed by using the model to predict the potency of the novel compounds.

Import required libraries

In [1]:
from pathlib import Path
from warnings import filterwarnings
import time

import pandas as pd
import numpy as np
from sklearn import svm, metrics, clone
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import auc, accuracy_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.Chem import PandasTools

import sys

from teachopencadd.utils import seed_everything
SEED = 22
seed_everything(SEED)


Define paths

In [2]:
HERE = Path(_dh[-1])
DATA = HERE / "data"

Read filtered ChEMBL bioactivity data from part 3

In [3]:
chembl_df = pd.read_csv(DATA/"BACE_compounds.csv",
    index_col=0,
)

print("Shape of dataframe : ", chembl_df.shape)
chembl_df.head()

Shape of dataframe :  (6691, 6)


,molecule_chembl_id,IC50,units,smiles,pIC50,graph
0,CHEMBL3969403,0.0002,nM,CC1(C)C(N)=N[C@](C)(c2cc(NC(=O)c3ccc(C#N)cn3)c...,12.698970,"Data(x=[30, 9], edge_index=[2, 64], edge_attr=..."
1,CHEMBL3937515,0.0009,nM,COc1cnc(C(=O)Nc2ccc(F)c([C@]3(C)CS(=O)(=O)C(C)...,12.045757,"Data(x=[30, 9], edge_index=[2, 64], edge_attr=..."
2,CHEMBL3949213,0.0010,nM,C[C@@]1(c2cc(NC(=O)c3ccc(C#N)cn3)ccc2F)CS(=O)(...,12.000000,"Data(x=[32, 9], edge_index=[2, 70], edge_attr=..."
3,CHEMBL3955051,0.0018,nM,CC1(C)C(N)=N[C@](C)(c2cc(NC(=O)c3cnc(C(F)F)cn3...,11.744727,"Data(x=[31, 9], edge_index=[2, 66], edge_attr=..."
4,CHEMBL3936264,0.0057,nM,C[C@@]1(c2cc(NC(=O)c3ccc(OC(F)F)cn3)ccc2F)CS(=...,11.244125,"Data(x=[30, 9], edge_index=[2, 64], edge_attr=..."


Keep only the needed columns

In [4]:
chembl_df = chembl_df[["molecule_chembl_id", "smiles", "pIC50"]]
chembl_df.head()

,molecule_chembl_id,smiles,pIC50
0,CHEMBL3969403,CC1(C)C(N)=N[C@](C)(c2cc(NC(=O)c3ccc(C#N)cn3)c...,12.698970
1,CHEMBL3937515,COc1cnc(C(=O)Nc2ccc(F)c([C@]3(C)CS(=O)(=O)C(C)...,12.045757
2,CHEMBL3949213,C[C@@]1(c2cc(NC(=O)c3ccc(C#N)cn3)ccc2F)CS(=O)(...,12.000000
3,CHEMBL3955051,CC1(C)C(N)=N[C@](C)(c2cc(NC(=O)c3cnc(C(F)F)cn3...,11.744727
4,CHEMBL3936264,C[C@@]1(c2cc(NC(=O)c3ccc(OC(F)F)cn3)ccc2F)CS(=...,11.244125


Add column for fingerprints

In [5]:
compound_df = chembl_df.copy()

Set the number of folds

In [6]:
N_FOLDS = 5

Fingerprints. The structures of the compounds will be transferred from SMILES notation to Morgan Fingerprints with a radius of 3, to explicitly describe the features of the compounds.

Define function to convert the smiles into fingerprints

In [7]:
def smiles_to_fp(smiles, method="maccs", n_bits=2048):
    """
    Encode a molecule from a SMILES string into a fingerprint.

    Parameters
    ----------
    smiles : str
        The SMILES string defining the molecule.

    method : str
        The type of fingerprint to use. Default is MACCS keys.

    n_bits : int
        The length of the fingerprint.

    Returns
    -------
    array
        The fingerprint array.

    """

    # convert smiles to RDKit mol object
    mol = Chem.MolFromSmiles(smiles)

    if method == "maccs":
        return np.array(MACCSkeys.GenMACCSKeys(mol))
    if method == "morgan2":
        return np.array(GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits))
    if method == "morgan3":
        return np.array(GetMorganFingerprintAsBitVect(mol, 3, nBits=n_bits))
    else:
        # NBVAL_CHECK_OUTPUT
        print(f"Warning: Wrong method specified: {method}. Default will be used instead.")
        return np.array(MACCSkeys.GenMACCSKeys(mol))

[Moet dat niet later pas?]

Use Morgan fingerprint with radius 3

In [8]:
compound_df["fp"] = compound_df["smiles"].apply(smiles_to_fp, args=("morgan3",))
compound_df.head()

,molecule_chembl_id,smiles,pIC50,fp
0,CHEMBL3969403,CC1(C)C(N)=N[C@](C)(c2cc(NC(=O)c3ccc(C#N)cn3)c...,12.698970,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1,CHEMBL3937515,COc1cnc(C(=O)Nc2ccc(F)c([C@]3(C)CS(=O)(=O)C(C)...,12.045757,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,CHEMBL3949213,C[C@@]1(c2cc(NC(=O)c3ccc(C#N)cn3)ccc2F)CS(=O)(...,12.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,CHEMBL3955051,CC1(C)C(N)=N[C@](C)(c2cc(NC(=O)c3cnc(C(F)F)cn3...,11.744727,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,CHEMBL3936264,C[C@@]1(c2cc(NC(=O)c3ccc(OC(F)F)cn3)ccc2F)CS(=...,11.244125,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


Define function for machine learning model training and validation in a cross-validation loop

In [9]:
def crossvalidation_reg(ml_model, df, n_folds=5, verbose=False):
    """
    Machine learning model training and validation in a cross-validation loop.

    Parameters
    ----------
    ml_model: sklearn model object
        The machine learning model to train.
    df: pd.DataFrame
        Data set with SMILES and their associated activity labels.
    n_folds: int, optional
        Number of folds for cross-validation.
    verbose: bool, optional
        Performance measures are printed.

    Returns
    -------
    None

    """
    t0 = time.time()
    # Shuffle the indices for the k-fold cross-validation
    kf = KFold(n_splits=n_folds, shuffle=True)#, random_state=SEED)

    # Results for each of the cross-validation folds
    MAE_per_fold = []
    RMSE_per_fold = []

    # Loop over the folds
    for train_index, test_index in kf.split(df):
        # clone model -- we want a fresh copy per fold!
        fold_model = clone(ml_model)
        # Training

        # Convert the fingerprint and the label to a list
        train_x = df.iloc[train_index].fp.tolist()
        train_y = df.iloc[train_index].pIC50.tolist()

        # Fit the model
        fold_model.fit(train_x, train_y)

        # Testing

        # Convert the fingerprint and the label to a list
        test_x = df.iloc[test_index].fp.tolist()
        test_y = df.iloc[test_index].pIC50.tolist()

        test_results = fold_model.predict(test_x)
        # Prediction probability on test set
        from sklearn import metrics

        MAE_per_fold.append(metrics.mean_absolute_error(test_y, test_results))
        #print('Mean Squared Error (MSE):', metrics.mean_squared_error(test_y, test_results))
        RMSE_per_fold.append(np.sqrt(metrics.mean_squared_error(test_y, test_results)))

        #from sklearn.metrics import matthews_corrcoef


        #print(matthews_corrcoef(test_y, test_results))
        #mape = np.mean(np.abs((gt - pred) / np.abs(gt)))
        #print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
        #print('Accuracy:', round(100*(1 - mape), 2))
    return(MAE_per_fold,RMSE_per_fold,fold_model)

Random forest regressor model

The benefit of a regression model as opposed to a classification model is that it can not only predict if the new compound is active or inactive, but also the pIC50 value of the compound.

In [10]:
models = [{"label": "Model_RF_reg", "model": RandomForestRegressor}]

Train model with RandomForestRegressor and show validation scores

In [11]:
regressor = RandomForestRegressor()
MAE, RMSE,trained_model = crossvalidation_reg(regressor, compound_df, n_folds=N_FOLDS)

print(
f"Mean Absolute Error (MAE): {np.mean(MAE):.2f} \t"
f"and std : {np.std(MAE):.2f} \n"
f"Root Mean Square Error (RMSE): {np.mean(RMSE):.2f} \t"
f"and std : {np.std(RMSE):.2f} \n"
)


Mean Absolute Error (MAE): 0.54 	and std : 0.01 
Root Mean Square Error (RMSE): 0.74 	and std : 0.01 



In [12]:
compound_df.head()

,molecule_chembl_id,smiles,pIC50,fp
0,CHEMBL3969403,CC1(C)C(N)=N[C@](C)(c2cc(NC(=O)c3ccc(C#N)cn3)c...,12.698970,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1,CHEMBL3937515,COc1cnc(C(=O)Nc2ccc(F)c([C@]3(C)CS(=O)(=O)C(C)...,12.045757,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,CHEMBL3949213,C[C@@]1(c2cc(NC(=O)c3ccc(C#N)cn3)ccc2F)CS(=O)(...,12.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,CHEMBL3955051,CC1(C)C(N)=N[C@](C)(c2cc(NC(=O)c3cnc(C(F)F)cn3...,11.744727,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,CHEMBL3936264,C[C@@]1(c2cc(NC(=O)c3ccc(OC(F)F)cn3)ccc2F)CS(=...,11.244125,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


To validate the performance of the model, the data was split into a train and test set. In this case this is done in a 5-fold cross-validation. Separate models are trained, each with one of these folds as test data, and all other folds as training data.
We then considered the overall performance. The errors between the predicted data and obtained experimental data are determined with a Mean Absolute Error (MAE) and a Root Mean Square Error (RMSE).


[Zijn deze waarde oke? Hoe kunnen ze verbeterd worden? Below 0.6 quite decent?]

In [13]:
print(trained_model)

RandomForestRegressor()


This trained model will below be used to predict the pIC50 values of the novel, drug-like, generated compounds of part 8.

### Generated Compounds: predict pIC50 with the trained random forest regression model

Read the filtered, generated compound data of part 8

In [14]:
novelcompound_df = pd.read_csv(
   DATA / "generated_part8.csv", index_col=0,
)

print("Shape of dataframe : ", novelcompound_df.shape)
novelcompound_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Jurren/Documents/GitHub/ACMDD/data/generated_part8.csv'

List with SMILES

In [ ]:
novelcompound_df_smiles = novelcompound_df.SMILES.tolist()

Convert SMILES to morgan fingerprints with radius 3. Use these fingerprints to predict the pIC50 of these compounds with the trained random forest regression model. 

In [ ]:
fps = []

for smiles in novelcompound_df_smiles: 
    fp = smiles_to_fp(smiles,'morgan3')
    fps.append(fp)

predictions = trained_model.predict(fps)

Show results

In [ ]:
novelcompound_df_smiles_results = pd.DataFrame({
"SMILES" : novelcompound_df_smiles,
"predicted_pIC50" : predictions
})

In [ ]:
novelcompound_df_smiles_results_m = pd.merge(
    novelcompound_df_smiles_results,
    novelcompound_df,
    on = "SMILES",
)

PandasTools.AddMoleculeColumnToFrame(novelcompound_df_smiles_results_m, smilesCol="SMILES")
novelcompound_df_smiles_results_m.sort_values(by="predicted_pIC50", ascending = False, inplace = True)
novelcompound_df_smiles_results_m.reset_index(drop=True, inplace=True)
novelcompound_df_smiles_results_m.head(10)

As can been seen in the table above, the all top 10 molecules fulfil Lipinski's rule of 5, which could mean that they are oral bioavailable. The molecule with the highest predicted pIC50 value is N-(3-(5-amino-3,6,6-trimethyl-1,1-dioxido-3,6-dihydro-2H-1,4-thiazin-3-yl)-4-fluorophenyl)-5-methoxypicolinamide. This molecule does not contain the scaffold of the largest cluster. However, it has the common substructure of the high potent inhibitors (threshold 0.7, with and without ring match). The predicted pIC50 of this compound is 11.37, which is quite high. Moreover, the molecule is not known in the PubChem database. Therefore, this can be a promising, novel inhibitor of BACE1. 

Save results of the novel compounds

In [ ]:
output_df = novelcompound_df_smiles_results_m.drop("ROMol", axis = 1)
output_df.to_csv(DATA/"nieuwmoleculenlijst_voorspellingen.csv")
output_df.head()